# Recomendador de peliculas

In [37]:
import pandas as pd
import numpy as np
import sqlite3



pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [38]:
url_movies = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv'

url_credits = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv'

In [39]:
df_movies = pd.read_csv(url_movies)

df_credits = pd.read_csv(url_credits)

df_movies.to_csv('../data/raw/movies.csv')
df_credits.to_csv('../data/raw/credits.csv')

In [40]:
df_movies.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
3608,5000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 36, ""name...",NaN,29996,"[{""id"": 536, ""name"": ""israel""}, {""id"": 1160, ""...",en,Solomon and Sheba,"Under the rule of King David, Israel is united...",0.835617,"[{""name"": ""Edward Small Productions"", ""id"": 13...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1959-12-25,12200000,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Only once in 3000 years...anything like...,Solomon and Sheba,5.4,8
598,75000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.warnerbros.com/rock-ages,80585,"[{""id"": 4344, ""name"": ""musical""}, {""id"": 8900,...",en,Rock of Ages,A small town girl and a city boy meet on the S...,23.515938,"[{""name"": ""New Line Cinema"", ""id"": 12}, {""name...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-06-13,59418613,123.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Nothin' but a good time,Rock of Ages,6.0,385
1693,12000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 27, ""name...",NaN,11249,"[{""id"": 1009, ""name"": ""baby""}, {""id"": 6300, ""n...",en,Seed of Chucky,The killer doll is back! The all-new film is t...,12.653831,"[{""name"": ""Rogue Pictures"", ""id"": 134}]","[{""iso_3166_1"": ""RO"", ""name"": ""Romania""}, {""is...",2004-11-11,24829644,87.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Fear The Second Coming,Seed of Chucky,4.9,287
1232,40000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.morningglorymovie.com/,38357,"[{""id"": 13281, ""name"": ""work ethic""}, {""id"": 1...",en,Morning Glory,A young and devoted morning television produce...,22.491388,"[{""name"": ""Bad Robot"", ""id"": 11461}, {""name"": ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2010-01-12,58785180,102.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Breakfast TV just got interesting.,Morning Glory,6.1,412
2576,14000000,"[{""id"": 27, ""name"": ""Horror""}]",NaN,12212,"[{""id"": 886, ""name"": ""film making""}, {""id"": 62...",en,Urban Legends: Final Cut,The making of a horror movie takes on a terrif...,8.885017,"[{""name"": ""Original Film"", ""id"": 333}, {""name""...","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}, {""iso...",2000-09-19,38574362,97.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Legends Never Die.,Urban Legends: Final Cut,4.4,100


In [41]:
df_credits.sample(5)

,movie_id,title,cast,crew
177,77950,Turbo,"[{""cast_id"": 5, ""character"": ""Turbo (voice)"", ...","[{""credit_id"": ""52fe4985c3a368484e12fb93"", ""de..."
4688,13363,The Man from Earth,"[{""cast_id"": 3, ""character"": ""John Oldman"", ""c...","[{""credit_id"": ""52fe45609251416c75054411"", ""de..."
3102,222649,Rosewater,"[{""cast_id"": 10, ""character"": ""Maziar Bahari"",...","[{""credit_id"": ""555347db92514162d10016e9"", ""de..."
3353,9392,The Descent,"[{""cast_id"": 1, ""character"": ""Sarah"", ""credit_...","[{""credit_id"": ""52fe44f1c3a36847f80b34cd"", ""de..."
1529,606,Out of Africa,"[{""cast_id"": 22, ""character"": ""Karen Christenc...","[{""credit_id"": ""52fe425cc3a36847f8018663"", ""de..."


Voy a comprobar si mis titulos son unicos, las instrucciones indican que deberia hacer un join a traves de la columna titulo, pero si hay algun titulo repetido, esto borraria y solo dejaria uno.

In [42]:
df_movies.shape

(4803, 20)

In [43]:
df_movies['title'].nunique()

4800

In [44]:
repeated_titles = df_credits['title'][df_credits['title'].duplicated()].unique()
print(repeated_titles)

['The Host' 'Out of the Blue' 'Batman']


Veo que si que tengo duplicados, voy a comprobar las filas a ver si son duplicados completos

In [45]:
resultado = df_movies[df_movies['title'] == 'Batman']
resultado

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
1359,35000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",NaN,268,"[{""id"": 848, ""name"": ""double life""}, {""id"": 84...",en,Batman,The Dark Knight of Gotham City begins his war ...,44.104469,"[{""name"": ""PolyGram Filmed Entertainment"", ""id...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",1989-06-23,411348924,126.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Have you ever danced with the devil in the pal...,Batman,7.0,2096
4267,1377800,"[{""id"": 10751, ""name"": ""Family""}, {""id"": 12, ""...",NaN,2661,"[{""id"": 339, ""name"": ""submarine""}, {""id"": 849,...",en,Batman,The Dynamic Duo faces four super-villains who ...,9.815394,"[{""name"": ""Twentieth Century Fox Film Corporat...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1966-07-30,0,105.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,He's Here Big As Life In A Real Bat-Epic,Batman,6.1,203


In [46]:
resultado = df_movies[df_movies['title'] == 'Out of the Blue']
resultado

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
3647,0,"[{""id"": 18, ""name"": ""Drama""}]",NaN,39269,"[{""id"": 4470, ""name"": ""punk""}, {""id"": 10183, ""...",en,Out of the Blue,Dennis Hopper is a hard-drinking truck driver ...,0.679351,"[{""name"": ""Robson Street"", ""id"": 71953}]","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}]",1980-05-01,0,94.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A harrowing drama from the director of Easy Rider,Out of the Blue,6.5,17
3693,0,"[{""id"": 18, ""name"": ""Drama""}]",NaN,10844,"[{""id"": 2658, ""name"": ""new zealand""}, {""id"": 3...",en,Out of the Blue,Ordinary people find extraordinary courage in ...,0.706355,[],"[{""iso_3166_1"": ""NZ"", ""name"": ""New Zealand""}]",2006-10-12,0,103.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The true story of a small town massacre,Out of the Blue,5.9,18


In [47]:
resultado = df_movies[df_movies['title'] == 'The Host']
resultado

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
972,44000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",NaN,72710,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,The Host,A parasitic alien soul is injected into the bo...,42.933027,"[{""name"": ""Nick Wechsler Productions"", ""id"": 8...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-03-22,63327201,125.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,You will be one of us,The Host,6.0,1817
2877,11000000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 18, ""nam...",http://www.hostmovie.com/,1255,"[{""id"": 1261, ""name"": ""river""}, {""id"": 1880, ""...",ko,괴물,Gang-du is a dim-witted man working at his fat...,27.655270,"[{""name"": ""Cineclick Asia"", ""id"": 685}, {""name...","[{""iso_3166_1"": ""KR"", ""name"": ""South Korea""}]",2006-07-27,88489643,119.0,"[{""iso_639_1"": ""ko"", ""name"": ""\ud55c\uad6d\uc5...",Released,Monsters are real.,The Host,6.7,537


Puedo observar que son completamente diferentes, por lo tanto unir por titulo no seria factible, ya que estos repetidos se eliminarian

Voy a comprobar duplicados en la columna ID, en caso de que no los haya sera la que utilice para unir mis dos datasets

In [48]:
print(df_movies['id'].nunique())
print(df_credits['movie_id'].nunique())

4803
4803


Puedo observar que no hay duplicados en los ids de cada uno de mis datasets, intentare hacer el join a traves de ellos, en caso de obtener mas filas de las que tengo originalmente, sera porque no coinciden entre si

In [49]:
# Creo mi database y la conexion a ella
conn = sqlite3.connect('../data/processed/movie_database.db')

# Creo el objeto cursor que interactuara con ella
cursor = conn.cursor()

# Creo mis tablas
df_movies.to_sql('tabla_movies', conn, if_exists='replace', index=False)
df_credits.to_sql('tabla_credits', conn, if_exists='replace', index=False)

4803

In [50]:
query = """
CREATE TABLE IF NOT EXISTS tabla_resultados AS 
SELECT tabla_credits.movie_id, tabla_movies.title, tabla_movies.overview, 
       tabla_movies.genres, tabla_movies.keywords, tabla_credits.cast, tabla_credits.crew
FROM tabla_movies
INNER JOIN tabla_credits
ON tabla_movies.id = tabla_credits.movie_id;
"""


cursor.execute(query)
conn.commit()

In [51]:
df = pd.read_sql('select * from tabla_resultados', conn)

conn.close()

In [52]:
df.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
1191,11551,Small Soldiers,When missile technology is used to enhance toy...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 12, ""nam...","[{""id"": 3599, ""name"": ""defense industry""}, {""i...","[{""cast_id"": 1, ""character"": ""Alan Abernathy"",...","[{""credit_id"": ""52fe44589251416c75031709"", ""de..."
2617,72359,Peeples,The story follows what happens when a child ps...,"[{""id"": 35, ""name"": ""Comedy""}]","[{""id"": 179431, ""name"": ""duringcreditsstinger""...","[{""cast_id"": 4, ""character"": ""Wade Walker"", ""c...","[{""credit_id"": ""52fe4866c3a368484e0f6c41"", ""de..."
3854,142061,"Batman: The Dark Knight Returns, Part 2",Batman has stopped the reign of terror that Th...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 16, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 2964...","[{""cast_id"": 3, ""character"": ""Batman / Bruce W...","[{""credit_id"": ""5612aa2ec3a368680b0159c9"", ""de..."
4180,58492,The Greatest Movie Ever Sold,"A documentary about branding, advertising and ...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 99, ""nam...","[{""id"": 3485, ""name"": ""comedian""}, {""id"": 1026...","[{""cast_id"": 8, ""character"": ""Himself"", ""credi...","[{""credit_id"": ""52fe4965c3a36847f8197e01"", ""de..."
1073,8987,The River Wild,"While on a family vacation, rafting expert Ga...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1261, ""name"": ""river""}, {""id"": 1621, ""...","[{""cast_id"": 1, ""character"": ""Gail Hartman"", ""...","[{""credit_id"": ""52fe44cfc3a36847f80ab353"", ""de..."


In [53]:
df.shape

(4803, 7)

Con esto he conseguido unir las dos bases de datos sin eliminar las filas repetidas.

nearest neighbours, tf-idf

In [54]:
import json

df = df.assign(genres=lambda x: x['genres'].apply(lambda y: [item['name'] for item in json.loads(y)] if pd.notna(y) else None))
df = df.assign(keywords=lambda x: x['keywords'].apply(lambda y: [item['name'] for item in json.loads(y)] if pd.notna(y) else None))
df = df.assign(cast=lambda x: x['cast'].apply(lambda y: [item['name'] for item in json.loads(y)][:3] if pd.notna(y) else None))

# columna crew
df = df.assign(crew=lambda x: x['crew'].apply(lambda y: next((item['name'] for item in json.loads(y) if item.get('job') == 'Director'), None) if pd.notna(y) else None))

df = df.assign(overview=lambda x: x['overview'].apply(lambda y: [str(y)] if pd.notna(y) else []))

In [55]:
df.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton


In [56]:
df.head(5)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton


In [57]:
from funciones import remove_spaces

# Borro los espacios en las columnas
df['genres'] = df['genres'].apply(remove_spaces)
df['cast'] = df['cast'].apply(remove_spaces)
df['crew'] = df['crew'].apply(remove_spaces)
df['keywords'] = df['keywords'].apply(remove_spaces)

In [58]:
df.head(5)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",JamesCameron
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",GoreVerbinski
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",SamMendes
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",ChristopherNolan
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",AndrewStanton


In [59]:
df["genres"] = df["genres"].apply(lambda x: [str(genre) for genre in x])
df["keywords"] = df["keywords"].apply(lambda x: [str(keyword) for keyword in x])
df["cast"] = df["cast"].apply(lambda x: [str(actor) for actor in x])

# Convierto mi columna 'crew' en una lista para poder hacer el .join
df["crew"] = df["crew"].apply(lambda x: [x] if pd.notna(x) and isinstance(x, str) else [])

In [60]:
df.head(5)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [61]:
df["tags"] = df["overview"] + df["genres"] + df["keywords"] + df["cast"] + df["crew"]
df["tags"] = df["tags"].apply(lambda x: ",".join(x).replace(",", " "))

In [62]:
df.drop(columns=["genres", "keywords", "cast", "crew", "overview"], inplace=True)

df.iloc[0].tags

'In the 22nd century  a paraplegic Marine is dispatched to the moon Pandora on a unique mission  but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [63]:
df.head()

,movie_id,title,tags
0,19995,Avatar,In the 22nd century a paraplegic Marine is di...
1,285,Pirates of the Caribbean: At World's End,Captain Barbossa long believed to be dead ha...
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,John Carter is a war-weary former military ca...


## Vectorizado

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer
from stop_words import get_stop_words
import pickle


stop_words = get_stop_words('en')

x = df['tags']

tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)
X_vectorized = tfidf_vectorizer.fit_transform(x)

with open('../models/tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(X_vectorized, f)

C:\Users\Carlos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn(


In [65]:
from sklearn.neighbors import NearestNeighbors

# cosine: mide similitud del angulo entre dos vectores, análisis de textos y datos de alta dimensión.
nn_model = NearestNeighbors(n_neighbors=5, metric='cosine', algorithm='brute', n_jobs=-1)

nn_model.fit(X_vectorized)

with open('../models/Nearest_neighbour.pkl', 'wb') as f:
    pickle.dump(nn_model, f)

In [ ]:
input_title = input('Write the title of the movie: ')

def get_movie_recommendations(movie_title, dataset, model, x):
    # Obtener el índice de la película
    movie_index = dataset[dataset["title"] == movie_title].index[0]
    
    # Encontrar los Nearest Neighbours
    distances, indices = model.kneighbors(x[movie_index], n_neighbors=6)
    
    # Hago la lista con los vecinos encontrados, excluyo el primer titulo en el output
    similar_movies = [(dataset["title"].iloc[i], distances[0][j]) for j, i in enumerate(indices[0])]

    return similar_movies[1:]

# Suponiendo que 'df' es tu DataFrame original y 'X_vectorized' la matriz TF-IDF
recommendations = get_movie_recommendations(input_title, df, nn_model, X_vectorized)

if recommendations:
    print("Film recommendations for '{}'".format(input_title))
    for movie, distance in recommendations:
        print("- Film: {}".format(movie))
else:
    print("I didnt find that title! Check spelling!")

I don't find that title!
No recommendations found.
